# LiteLLM Tool calling capabilities with Unity Catalog

## Prerequisites

**API Key**
To run this tutorial, you will need an OpenAI API key. 

Once you have acquired your key, set it to the environment variable `OPENAI_API_KEY`.

Below, we validate that this key is set properly in your environment.

**Packages**

To interface with both UnityCatalog and LiteLLM, you will need to install the following packages:

```
pip install litellm unitycatalog-client unitycatalog-litellm -q -U
```

In [0]:
import os

assert "OPENAI_API_KEY" in os.environ, (
    "Please set the OPENAI_API_KEY environment variable to your OpenAI API key"
)

## Configuration and Client setup

In order to connect to your Unity Catalog server, you'll need an instance of the `ApiClient` from the `unitycatalog-client` package. 

> Note: If you don't already have a Catalog and a Schema created, be sure to create them before running this notebook and adjust the `CATALOG` and `SCHEMA` variables below to suit.

In [0]:
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.client import ApiClient, Configuration

config = Configuration(host="http://localhost:8080/api/2.1/unity-catalog")

api_client = ApiClient(configuration=config)

client = UnitycatalogFunctionClient(api_client=api_client)

# Replace with your own catalog and schema
CATALOG = "AICatalog"
SCHEMA = "AISchema"

## Define functions and register them to Unity Catalog

In this next section, we'll be defining a Python function and creating it within Unity Catalog so that it can be retrieved and used as tools within our LiteLLM call. 

There are a few things to keep in mind when creating functions for use with the `create_python_function` API:

- Ensure that your have properly defined types for all arguments and for the return of the function.
- Ensure that you have a Google-style docstring defined that includes descriptions for the function, each argument, and the return of the function. This is critical, as these are used to populate the metadata associated with the function within Unity Catalog, providing contextual data for an LLM to understand when and how to call the tool associated with this function.
- If there are packages being called that are not part of core Python, ensure that the import statements are locally scoped (defined within the function body).

In [0]:
# Define our function
def get_weather(location: str) -> str:
    """
    Query the weather and return a string with the current temperature.

    Args:
      location: The location to get the weather

    """
    return "Hot!"


# Register functions
function_info = client.create_python_function(
    func=get_weather, catalog=CATALOG, schema=SCHEMA, replace=True
)
execution_function_name = function_info.full_name

## Create a Toolkit instance of the functions

Now that the functions have been created within Unity Catalog, we can use the `unitycatalog-litellm` package to create a toolkit instance that our Agent will 'understand' as valid tools to use within its APIs. 

In [0]:
from unitycatalog.ai.litellm.toolkit import UCFunctionToolkit

toolkit = UCFunctionToolkit(function_names=[execution_function_name])
tools = toolkit.tools
tools

[LiteLLMTool(name='main__default__get_weather', description='Query the weather and return a string with the current temperature.', tool={'type': 'function', 'function': {'name': 'main__default__get_weather', 'strict': True, 'parameters': {'properties': {'location': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'description': 'The location to get the weather', 'title': 'Location'}}, 'title': 'main__default__get_weather__params', 'type': 'object', 'additionalProperties': False, 'required': ['location']}, 'description': 'Query the weather and return a string with the current temperature.'}})]

## Pass a Tool Definition to LiteLLM

Now that we have a UCFunctionToolkit with definitions of our tool, we can leverage them in LiteLLM.

In [0]:
import litellm

# Define your request
question = "What is the current temperature in SF?"
messages = [{"role": "user", "content": question}]

# Show the response
response = litellm.completion(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
response

 ModelResponse(id='chatcmpl-Ae2ploBk7zcxPh16PHWaJXJiI9qRa', created=1734107377, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_6fc10e10eb', choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"location":"San Francisco"}', name='main__default__get_weather'), id='call_KVNXgt8RRqUjug7iaTnOQD7d', type='function')], function_call=None))], usage=Usage(completion_tokens=19, prompt_tokens=84, total_tokens=103, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier=None)


## Call the UC Function
Based on the above response, LiteLLM wants to call the tool. Let's execute the function via the `generate_tool_call_messages` method.

In [0]:
from unitycatalog.ai.litellm.utils import generate_tool_call_messages

tool_messages = generate_tool_call_messages(
    response=response, client=client, conversation_history=messages
)

response_2 = litellm.completion(
    model="gpt-4o-mini",
    messages=tool_messages,
)
response_2


 ModelResponse(id='chatcmpl-Ae2povzf9i8bRpqwCRXBq3YsbP2kH', created=1734107380, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_6fc10e10eb', choices=[Choices(finish_reason='stop', index=0, message=Message(content='The current temperature in San Francisco is hot! For the exact temperature, you might want to check a reliable weather website or app.', role='assistant', tool_calls=None, function_call=None))], usage=Usage(completion_tokens=26, prompt_tokens=48, total_tokens=74, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier=None)
